In [2]:
# %load test_2.py

"根据官方教材来解题"

#data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import math

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#machine leanring
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [136]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

In [179]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# train_df,test_df,Y_train,Y_test = train_test_split(train_df,train_df.Survived,random_state=0)
# test_df = test_df.drop('Survived',axis=1)


In [160]:

# 熟悉数据
# print(train_df.columns.values)

#preview the data
# train_df.head()
#train_df.tail()
# train_df.describe()

# train_df[train_df.Name.str.contains('Lady')].Name

# # 年龄为空（NaN）的人
# train_df[train_df.Age != train_df.Age]
# train_df[train_df.Embarked != train_df.Embarked]

In [153]:

# combine[0][["Title", "Survived"]].groupby(['Title'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# combine[0][["Age", "Survived"]].groupby(['Age'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# combine[0][["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# combine[0][["IsAlone", "Survived"]].groupby(['IsAlone'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# combine[0][["Fare", "Survived"]].groupby(['Fare'], as_index=False).mean().sort_values(by='Survived', ascending=False)
combine[0][["Pclass", "Survived"]].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

# # 两种Survived的年龄分布直方图
# g = sns.FacetGrid(train_df, col='Survived')
# g.map(plt.hist, 'Age', bins=20)

# # 是否Survived和三种Pclass的年龄分布直方图
# # grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
# grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', size=2.2, aspect=1.6)
# grid.map(plt.hist, 'Age', alpha=.5, bins=20)
# grid.add_legend();

# # 两种性别和三个Embarked（登船港口）的生存率关系图
# # grid = sns.FacetGrid(train_df, col='Embarked')
# grid = sns.FacetGrid(train_df, row='Embarked', size=2.2, aspect=1.6)
# grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
# grid.add_legend()

# # 是否Survived和三个Embarked（登船港口）的性别分布直方图
# # grid = sns.FacetGrid(train_df, col='Embarked', hue='Survived', palette={0: 'k', 1: 'w'})
# grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', height=2.2, aspect=1.6)
# grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None,order=None)
# grid.add_legend()

# # 三种船舱和两种性别的年龄直方图
# # grid = sns.FacetGrid(train_df, col='Pclass', hue='Gender')
# grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', size=2.2, aspect=1.6)
# grid.map(plt.hist, 'Age', alpha=.5, bins=20)
# grid.add_legend()

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [176]:
# 删除Ticket和Cabin，暂时认为船票信息和船舱信息与生存率无关
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)

combine = [train_df,test_df]

In [180]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Mr        228
Miss      101
Mrs        58
Master     23
Dr          3
Rev         3
Mme         1
Don         1
Name: Title, dtype: int64

In [177]:
# ①处理Name->新增feture Title
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
# 每个称谓的分布情况
pd.crosstab(train_df['Title'], train_df['Sex'])

# 把称谓进行分类。根据称谓的社会地位，把稀有的称谓，且无法区分性别的再分成两种Upper（上流人士）
for dataset in combine:
# 1.Don 基督教男子教名。1名->Mr；
# 2.Dona 女士。->Miss
# 3.Lady 和 Sir 可以认为是英国皇家中男女的称呼。在数据集中有2名，且他们是夫妇（Duff Gordon 夫妇），皇家爵位是男爵；
# 4.Countess 皇家称为，伯爵夫人或女伯爵。在数据集中有1名；
# 5.Capt 船长; captain （陆军）上尉; （海军） 上校。1名，在数据集中他是陆军上尉；
# 6.Col 上校。2名；
# 7.Major 陆军少校。2名；
# 8.Dr 博士,医生。6名；
# 9.Rev 牧师。6名；
# 10.Jonkheer 低级贵族。1名；

#     dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
#  	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Don','Mr')
    
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Dona','Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
    dataset['Title'] = dataset['Title'].replace('Dr','Master')
    dataset['Title'] = dataset['Title'].replace('Rev','Master')
    dataset['Title'] = dataset['Title'].replace('Jonkheer','Master')
    
    dataset['Title'] = dataset['Title'].replace('Lady','Upper')
    dataset['Title'] = dataset['Title'].replace('Sir','Upper')
    dataset['Title'] = dataset['Title'].replace('Countess','Upper')
    dataset['Title'] = dataset['Title'].replace('Capt','Upper')
    dataset['Title'] = dataset['Title'].replace('Col','Upper')
    
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

# 根据Name 把 Title分成5类
title_mapping = {"Mr": 0,"Master": 1, "Upper": 2, "Miss": 3, "Mrs": 3,"Lady":3}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(1)

train_df.head()

#删除Name和PassengerId属性
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop('Name', axis=1)
combine = [train_df, test_df]


In [178]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,0.0
1,1,1,female,38.0,1,0,71.2833,C,3.0
2,1,3,female,26.0,0,0,7.9250,S,3.0
3,1,1,female,35.0,1,0,53.1000,S,3.0
4,0,3,male,35.0,0,0,8.0500,S,0.0


In [163]:
# ②处理Sex。把性别转为 1和0
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'male': 0,'female': 1} ).astype(int)

train_df.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,S,0.0
1,1,1,1,38.0,1,0,71.2833,C,3.0
2,1,3,1,26.0,0,0,7.9250,S,3.0
3,1,1,1,35.0,1,0,53.1000,S,3.0
4,0,3,0,35.0,0,0,8.0500,S,0.0


In [164]:
# ③处理Age。
# 对年龄为空的样本，设置年龄（这里用中位数的方法）
guess_ages = np.zeros((2,3))
# guess_ages

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)
            
            #取中位数
            age_guess = guess_df.median()
            
            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int(age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

# train_df.head()

# 新增feture AgeBand，把年龄从低到高分为5个区间
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

# 根据5个年龄段，把各个年龄替换为0~4
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 3
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 2
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 0
# train_df.head()

# 删除AgeBand属性
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,1,1,0,7.2500,S,0.0
1,1,1,1,2,1,0,71.2833,C,3.0
2,1,3,1,1,0,0,7.9250,S,3.0
3,1,1,1,2,1,0,53.1000,S,3.0
4,0,3,0,2,0,0,8.0500,S,0.0


In [165]:
# ④处理SibSp和Parch。
# 新增feture FamilySize，家庭成员数量与存活率的关系
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)


# 新增feture IsAlone，是否独自和存活率的关系
for dataset in combine:
    dataset['IsAlone'] = 1
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 0

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

# 这里删除家庭成员相关属性
train_df = train_df.drop(['Parch', 'SibSp','FamilySize' ], axis=1)
test_df = test_df.drop(['Parch', 'SibSp','FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,1,7.2500,S,0.0,1
1,1,1,1,2,71.2833,C,3.0,1
2,1,3,1,1,7.9250,S,3.0,0
3,1,1,1,2,53.1000,S,3.0,1
4,0,3,0,2,8.0500,S,0.0,0


In [166]:
# # 年龄和船舱的乘积作为新feture，好像没看出有什么用
# for dataset in combine:
#     dataset['Age*Class'] = dataset.Age * dataset.Pclass

# train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

In [167]:
# ⑤处理Embarked。
# 查出最多的登船港口
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

# 用最多出现的Embarked填充Embarked为空的数据
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

# 把Embarked转化为数字特征
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'Q': 1, 'C': 2} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,1,7.2500,0,0.0,1
1,1,1,1,2,71.2833,2,3.0,1
2,1,3,1,1,7.9250,0,3.0,0
3,1,1,1,2,53.1000,0,3.0,1
4,0,3,0,2,8.0500,0,0.0,0


In [168]:
# ⑥处理Fare。
# 用Fare中间数填充Fare为空的数据
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

# 新增FareBand，票价区间
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

# 根据票价区间，把Fare替换为0~3四类
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone
0,0,3,0,1,0,0,0.0,1
1,1,1,1,2,3,2,3.0,1
2,1,3,1,1,1,0,3.0,0
3,1,1,1,2,3,0,3.0,1
4,0,3,0,2,1,0,0.0,0
5,0,3,0,1,1,1,0.0,0
6,0,1,0,2,3,0,0.0,0
7,0,3,0,3,2,0,1.0,1
8,1,3,1,1,1,0,3.0,1
9,1,2,1,3,2,2,3.0,1


In [169]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 7), (891,), (418, 7))

In [13]:
# logreg = LogisticRegression()
# logreg.fit(X_train,Y_train)
# Y_pred = logreg.predict(X_test)
# acc_log = logreg.score(X_train,Y_train)
# acc_log

In [14]:
# svc = SVC()
# svc.fit(X_train, Y_train)
# Y_pred = svc.predict(X_test)
# acc_svc = svc.score(X_train, Y_train)

In [15]:
# knn = KNeighborsClassifier(n_neighbors = 3)
# knn.fit(X_train, Y_train)
# Y_pred = knn.predict(X_test)
# acc_knn  = knn.score(X_train, Y_train)

In [16]:
# gaussian = GaussianNB()
# gaussian.fit(X_train, Y_train)
# Y_pred = gaussian.predict(X_test)
# acc_gaussian = gaussian.score(X_train, Y_train)

In [17]:
# perceptron = Perceptron()
# perceptron.fit(X_train, Y_train)
# Y_pred = perceptron.predict(X_test)
# acc_perceptron = perceptron.score(X_train, Y_train)

In [18]:
# linear_svc = LinearSVC()
# linear_svc.fit(X_train, Y_train)
# Y_pred = linear_svc.predict(X_test)
# acc_linear_svc = linear_svc.score(X_train, Y_train)

In [19]:
# sgd = SGDClassifier()
# sgd.fit(X_train, Y_train)
# Y_pred = sgd.predict(X_test)
# acc_sgd = sgd.score(X_train, Y_train)

In [20]:
# decision_tree = DecisionTreeClassifier()
# decision_tree.fit(X_train, Y_train)
# Y_pred = decision_tree.predict(X_test)
# acc_decision_tree = decision_tree.score(X_train, Y_train)

In [170]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = random_forest.score(X_train, Y_train)

In [171]:
# models = pd.DataFrame({
#     'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
#               'Random Forest', 'Naive Bayes', 'Perceptron', 
#               'Stochastic Gradient Decent', 'Linear SVC', 
#               'Decision Tree'],
#     'Score': [acc_svc, acc_knn, acc_log, 
#               acc_random_forest, acc_gaussian, acc_perceptron, 
#               acc_sgd, acc_linear_svc, acc_decision_tree]})
# models.sort_values(by='Score', ascending=False)


In [172]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })

In [173]:
submission.to_csv('result/submission_random_forest.csv', index=False)